# Scikit Learn - Pre-procesado de datos

- - -

Para esta ocación trabajaremos con un dataset que es un subconjunto creado por Barry Becker a partir de la base de datos del censo de 1994.

- - -
+ age
+ workclass
+ fnlwgt
+ education
+ education-num
+ marital-status
+ occupation
+ relationship
+ race
+ sex
+ capital-gain
+ capital-loss
+ hours-per-week
+ native-country


+ income

"fnlwgt es número de unidades en la población objetivo que representa la fila del dataset"
- - -

In [ ]:
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings(action="ignore")

In [ ]:
censo = pd.read_csv("https://raw.githubusercontent.com/4data-lab/datasets/master/adult.csv")

In [ ]:
censo.head()

- - -

In [ ]:
columns_list = ["age", "workclass", "fnlwgt", "education", "education-num", 
           "marital-status", "occupation", "relationship", "race", "sex", 
           "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"]

In [ ]:
censo = pd.read_csv("https://raw.githubusercontent.com/4data-lab/datasets/master/adult.csv", header=None, names=columns_list)

In [ ]:
censo.head()

In [ ]:
censo.shape

- - -

In [ ]:
print("Edad máxima del dataset: "+ str(censo["age"].max()))

In [ ]:
print("Edad mínima del dataset: "+ str(censo["age"].min()))

- - -

### Outliers

<img src=https://i.ibb.co/T8pTYm0/01.png width="250">

- - -

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
#¿Hay outliers en la edad del dataset?
sns.boxplot(x=censo["age"])

In [ ]:
plt.figure(figsize=(20,3))
plt.xticks(range(0,100))
sns.boxplot(x=censo["age"])
plt.show()

- - -
En el gráfico de boxplot podemos apreciar el valor mínimo (17) y el valor máximo (90)
- - -

- - -
Recordad, ¿cómo podemos averiguar cuáles son los cortes para determinar un outlier?

    corte inferior = primer cuartil - (rango intercuartil * 1.5)

    corte superior = tercer cuartil + (rango intercuartil * 1.5)
- - -

<img src=https://i.ibb.co/Kb5RJbv/02.png width="700">

- - -

In [ ]:
Q1 = censo["age"].quantile(0.25)
print(Q1)
#También se podría haber calculado con numpy de la siguiente manera:

#import numpy as np
#q1 = np.percentile(censo["age"], 25)

- - -

In [ ]:
Q3 = censo["age"].quantile(0.75)
print(Q3)
#También se podría haber calculado con numpy de la siguiente manera:

#import numpy as np
#q3 = np.percentile(censo["age"], 75)

- - -

<img src=https://i.ibb.co/tqzbQsW/03.gif width="250">

- - -

Recordad que el rango intercuartil es Q3 - Q1

In [ ]:
IQR = Q3 - Q1
print(IQR)

In [ ]:
corte_inferior = Q1 - (IQR * 1.5)
corte_superior = Q3 + (IQR * 1.5)

In [ ]:
corte_inferior

In [ ]:
corte_superior

- - -
Nuestro datos contienen valores mayores a 78, pero no disponen de valores menores que -2.

En el supueso caso que quisieramos aplicar dichos cortes a nuestro dataset podria hacerlo de la siguiente manera
- - -

In [ ]:
censo_filtrado = censo[(censo["age"] >= -2) & (censo["age"] <= 78)]

In [ ]:
censo_filtrado.head()

Otras opciones podrían ser: no eliminar estos valores atípicos, eliminarlos con un criterio del ámbito o negocio, etc.

Por ejemplo, en EEUU la edad mínima para poder trabajar con todos los derechos es 18 años.

La edad para obtener una jubilación completa es a los 67 años.

- - -

In [ ]:
#¿Qué tipo de income posible hay en el dataset?
censo["income"].value_counts()

In [ ]:
#Vamos a ver cuánta gente cobra más de 50k
censo[censo["income"] == ">50K"]

- - -

In [ ]:
#Vaya, el resultado anterior no ha proporcionado ningún valor. ¿Qué ha sucedido?
censo["income"][0] #Vemos que hay un espacio indeseado

In [ ]:
censo["income"][7]

- - -

In [ ]:
#recordad que con .columns podemos ver las columnas del dataset
censo.columns

In [ ]:
#printemos una instancia de cada columna para ver dónde hay espacios
for i in censo.columns:
    #Vamos a ver la primera fila campo a campo
    print(censo[i][0])

- - -

In [ ]:
censo.dtypes

- - -

Para eliminar los los caracteres iniciales y finales:
```
.str.strip 
```

Para eliminar los caracteres iniciales:
```
.str.lstrip
```


Para eliminar los caracteres finales:
```
.str.rstrip
```

Más información:

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.strip.html

- - -

In [ ]:
ejemplo = pd.DataFrame(["   Hola", "Mundo!   ", " Hola Mundo! ", "HELLO IT'S ME"], columns=["COLUMN"])

In [ ]:
print(ejemplo["COLUMN"])

In [ ]:
ejemplo["COLUMN"].str.strip()

- - -

In [ ]:
#Eliminamos definitivamente los espacios de nuestro dataset.
for i in censo.columns:
    if censo[i].dtype == "object":
        censo[i] = censo[i].str.strip()



- - -

In [ ]:
#comprovamos que efectivamente se han eliminado los espacios.
for i in censo.columns:
    print(censo[i][0])

- - -

In [ ]:
censo["income"][0]

In [ ]:
censo["income"][7]

- - -

In [ ]:
#Ahora, vamos a pasar nuestras labels de income de tipo object a números binarios
#(1 o 0)
#Otro método de hacerlo sería utilizando la función replace que ya conocemos
#censo["income"].replace({"<=50K":0,">50K":1})

In [ ]:
from sklearn import preprocessing

In [ ]:
le = preprocessing.LabelEncoder()

In [ ]:
le.fit(censo["income"])
censo["income"] = le.transform(censo["income"])

In [ ]:
censo["income"].value_counts()

- - -

In [ ]:
#Vemos qué opciones hay en la columna de raza.
censo["race"].value_counts()

- - -
### Dummies
<img src=https://i.ibb.co/mq2pWk9/04.png width="500">


In [ ]:
#Transformamos la raza en dummies
dummies = pd.get_dummies(censo["race"])

In [ ]:
dummies.head()

In [ ]:
#Añadimos los dummies en el dataset
censo = pd.concat([censo, dummies], axis=1)

In [ ]:
censo.head()

In [ ]:
#eliminamos la variable original de raza porque ya no nos interesa
censo = censo.drop("race", axis=1)

In [ ]:
censo.head(0)

- - -

In [ ]:
#Analicemos ahora la feature de relationship y vemos que también hay varias opciones.
censo["relationship"].value_counts()

In [ ]:
print('Los diferentes tipos de relaciones són:' + str(censo["relationship"].unique()))
print('Así que hay un total de {} categorías.' .format(censo["relationship"].nunique()))


- - -

In [ ]:
#Evaluamos cuántas cateogrías hay para cada tipo de variable
for i in censo.columns:
    if censo[i].dtypes == "object":
        número_categorias = censo[i].nunique()
        print(i, número_categorias)

- - -

In [ ]:
#Vemos que hay muchos países nativos.
censo["native-country"].value_counts()

In [ ]:
#Vamos a filtrar y hacer únicamente dos categorías: USA o OTROS.
def countries(x):
    if x == "United-States":
        return "country-usa"
    else:
        return "country-other"

In [ ]:
censo["native-country"] = censo["native-country"].apply(countries)

In [ ]:
censo["native-country"].value_counts()

- - -
Alternativamente, podríamos usar una funciones lambda como se indica a continuación:

In [ ]:
#censo["native-country"] = censo["native-country"].apply(lambda x: "country-usa" if x=="United-States" else "country-other")
#La diferencia entre Lambdas y List Comprehension estás ultimas se utilizan para crear listas, 
#las lambdas son funciones que pueden procesar como otras funciones y por lo tanto realizar un return de valores o listas

In [ ]:
#Ahora pasamos todas las columnas del dataset de tipo object a dummies
for i in censo.columns:
    if censo[i].dtypes == "object":
        dummies = pd.get_dummies(censo[i])
        censo = pd.concat([censo, dummies], axis=1)
        censo = censo.drop(i, axis=1)

In [ ]:
censo.shape

- - -

In [ ]:
#Dividimos entre X e y teniendo en cuenta que nuestro target va a ser averiguar 
#quién cobra más de 50K.
X = censo.drop("income", 1)
y = censo["income"]

- - -
### Escalado

En general, los algoritmos de aprendizaje automático se benefician de la estandarización del conjunto de datos. 

La estandarización de conjuntos de datos es un ejercicio común para muchos modelos de aprendizaje automático.

En esta ocacion usaremos el MinMaxScaler, escalaremos las características para que se encuentren entre un valor mínimo y máximo dado, a menudo entre cero y uno. 
- - -

\begin{align}
\Large X_{sc} = \frac{X - X_{min}}{X_{max} - X_{min}}
\end{align}

<img src=https://i.ibb.co/9TWvjt5/05.png width="700">

In [ ]:
#Vamos a pre-procesar para tener datos entre 0 y 1.
from sklearn.preprocessing import MinMaxScaler
X_maxmin = MinMaxScaler().fit_transform(X)

In [ ]:
#Vamos a pre-procesar para tener los datos estandarizados.
from sklearn.preprocessing import StandardScaler
X_norm = StandardScaler().fit_transform(X)

- - -

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#split de los datos SIN escalar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
#split de los datos escalados
X_train_maxmin, X_test_maxmin, y_train_maxmin, y_test_maxmin = train_test_split(X_maxmin, y, test_size=0.30, random_state=1)
#split de los datos normalizados
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(X_norm, y, test_size=0.30, random_state=1)

In [ ]:
LR = LogisticRegression()

In [ ]:
s1 = cross_val_score(LR, X_train, y_train, cv=10, scoring="roc_auc").mean()
print("AUC - Datos de validation SIN escalar: " + str(s1))
s2 = cross_val_score(LR, X_train_maxmin, y_train_maxmin, cv=10, scoring="roc_auc").mean()
print("AUC - Datos de validation escalados: " + str(s2))
s3 = cross_val_score(LR, X_train_n, y_train_n, cv=10, scoring="roc_auc").mean()
print("AUC - Datos de validation normalizados: "+ str(s3))


- - -